In [7]:
from collections import defaultdict
import cv2
import numpy as np
from ultralytics import YOLO

# Load the YOLOv8 models
tracking_model = YOLO('yolov8n-pose.pt')  # For tracking
pose_model = YOLO('C:\\Users\\sakar\\Semester Project\\semester-project-yolov8\\runs\\pose\\train22\weights\\last.pt')  # For pose detection, replace with your pose model path

# Open the video file
video_path = r"C:\Users\sakar\Semester Project\Spot Datasets\Recordings\movie_001.mp4"
cap = cv2.VideoCapture(video_path)

# Store the track history
track_history = defaultdict(lambda: [])

while cap.isOpened():
    # Read a frame from the video
    success, frame = cap.read()

    if success:
        # Run YOLOv8 tracking on the frame, persisting tracks between frames
        tracking_results = tracking_model.track(frame, persist=True)

        # Run pose detection on the same frame
        pose_results = pose_model(frame)[0]

        # Get the boxes and track IDs for tracking
        boxes = tracking_results[0].boxes.xywh.cpu()
        track_ids = tracking_results[0].boxes.id.int().cpu().tolist()

        # Visualize tracking results on the frame
        annotated_frame = tracking_results[0].plot()

        # Plot the tracks
        for box, track_id in zip(boxes, track_ids):
            x, y, w, h = box
            track = track_history[track_id]
            track.append((float(x), float(y)))  # x, y center point
            if len(track) > 30:  # Retain tracks for 30 frames
                track.pop(0)

            # Draw the tracking lines
            points = np.hstack(track).astype(np.int32).reshape((-1, 1, 2))
            cv2.polylines(annotated_frame, [points], isClosed=False, color=(230, 230, 230), thickness=2)

        # Draw pose keypoints for each detected person
        for result in pose_results:
            keypoints = result.keypoints.xy.tolist()
            for keypoint_index, (x, y) in enumerate(keypoints[0]):
                if x != 0 and y != 0:
                    cv2.putText(annotated_frame, str(keypoint_index), (int(x), int(y)),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        # Display the annotated frame with both tracking and pose keypoints
        cv2.imshow("YOLOv8 Tracking and Pose Detection", annotated_frame)

        # Break the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
    else:
        # End of the video
        break

# Release the video capture object and close the display window
cap.release()
cv2.destroyAllWindows()


requirements: Ultralytics requirement ['lapx>=0.5.2'] not found, attempting AutoUpdate...

requirements: AutoUpdate success ✅ 2.8s, installed 1 package: ['lapx>=0.5.2']
requirements: ⚠️ Restart runtime or rerun command for updates to take effect



ModuleNotFoundError: No module named 'lap'

In [5]:
!pip uninstall lapx -y

Found existing installation: lapx 0.5.5
Uninstalling lapx-0.5.5:
  Successfully uninstalled lapx-0.5.5


In [6]:
!pip install lapx>=0.5.2